In [3]:
#Use gmap to find the missing locations
import pandas as pd
import os
import csv
import pandas as pd
from math import radians, cos, sin ,tan
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import numpy as np
from urllib import parse

In [4]:
#initialise webdriver
from webdriver_manager.chrome import ChromeDriverManager
browser = webdriver.Chrome(ChromeDriverManager().install())
#If your webdriver matches browser's version, then skip this step and call the driver directly.

[WDM] - Downloading: 100%|██████████| 6.46M/6.46M [00:00<00:00, 30.6MB/s]
C:\Users\user\AppData\Local\Temp/ipykernel_15772/4089053854.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
#from https://tylerastro.medium.com/twd97-to-longitude-latitude-dde820d83405
#transform coordination into Taiwanese system
def lonlat_to_97( lon,lat):
    """
    It transforms longitude, latitude to TWD97 system.

    Parameters
    ----------
    lon : float
        longitude in degrees
    lat : float
        latitude in degrees 

    Returns
    -------
    x, y [TWD97]
    """
    
    lat = radians(lat)
    lon = radians(lon)
    
    a = 6378137.0
    b = 6356752.314245
    long0 = radians(121)
    k0 = 0.9999
    dx = 250000

    e = (1-b**2/a**2)**0.5
    e2 = e**2/(1-e**2)
    n = (a-b)/(a+b)
    nu = a/(1-(e**2)*(sin(lat)**2))**0.5
    p = lon-long0

    A = a*(1 - n + (5/4.0)*(n**2 - n**3) + (81/64.0)*(n**4  - n**5))
    B = (3*a*n/2.0)*(1 - n + (7/8.0)*(n**2 - n**3) + (55/64.0)*(n**4 - n**5))
    C = (15*a*(n**2)/16.0)*(1 - n + (3/4.0)*(n**2 - n**3))
    D = (35*a*(n**3)/48.0)*(1 - n + (11/16.0)*(n**2 - n**3))
    E = (315*a*(n**4)/51.0)*(1 - n)

    S = A*lat - B*sin(2*lat) + C*sin(4*lat) - D*sin(6*lat) + E*sin(8*lat)

    K1 = S*k0
    K2 = k0*nu*sin(2*lat)/4.0
    K3 = (k0*nu*sin(lat)*(cos(lat)**3)/24.0) * (5 - tan(lat)**2 + 9*e2*(cos(lat)**2) + 4*(e2**2)*(cos(lat)**4))

    y_97 = K1 + K2*(p**2) + K3*(p**4)

    K4 = k0*nu*cos(lat)
    K5 = (k0*nu*(cos(lat)**3)/6.0) * (1 - tan(lat)**2 + e2*(cos(lat)**2))

    x_97 = K4*p + K5*(p**3) + dx
    return x_97, y_97

In [47]:
#try
lonlat_to_97(120.14727,23.0510545102663)

(162615.35860108832, 2550191.1858210866)

In [34]:
'''
This code will put address into google map. 
The result shall be their coordination.
Though my best effort, there could be address errors that need to be corrected manually.
User has to open the driver first and run this code at google map.
It will send addresses into the map and read the coordination from url.
The addresses from gmap can be name of the store or schools instead of the real address.
'''
def addgmap(inaddress):
    cuurl = browser.current_url
    qry = browser.find_element_by_id("searchboxinput")
    qry.clear()
    qry.send_keys(inaddress)
    element = browser.find_element_by_id("searchbox-searchbutton") 
    element.click()
    time.sleep(7)
    url = browser.current_url
    if "/search/" in url:
        return np.nan,np.nan,np.nan,np.nan
    url1 = "0"
    if '!3d' in url:
        url1 = url.split('!3d')[1]
    address = ''
    address = url.split('place/')[1].split('/@')[0]
    address = parse.unquote(address)
    y,x =  0,0
    if url1 != "0":
        y,x = url1.split('!4d')[0],url1.split('!4d')[1][0:9]
    
    return url,x,y,address

In [134]:
#manual trial
url, x,y, address = addgmap('桃園市龜山區山鶯路200號')
x,y = float(x),float(y)
x,y = lonlat_to_97(x,y)
print(x,y)

C:\Users\user\AppData\Local\Temp/ipykernel_15772/596618866.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  qry = browser.find_element_by_id("searchboxinput")
C:\Users\user\AppData\Local\Temp/ipykernel_15772/596618866.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = browser.find_element_by_id("searchbox-searchbutton")


282930.2647536799 2764352.558596563


In [ ]:
#set the driver and import missing list
browser.get("https://www.google.com.tw/maps/")
df1 = pd.read_csv('add_missing_utf8.csv')
df1["Address"] = df1["Address"].str.split('號').str[0]+'號'#df1.Response_Address.isna()
df1[:20]

In [44]:
#If it crash, skip crashed one and start again from the next entry of the one failed.
for i in range(len(df1)):
    print(i,df1.loc[i,"Response_X"])
    time.sleep(5*random.random())
    url, x,y, address = addgmap(df1.loc[i,"Address"])
    x,y = float(x),float(y)
    x,y = lonlat_to_97(x,y)
    df1.loc[i,"Response_X"], df1.loc[i,"Response_Y"] = x,y
    df1.loc[i,"Response_Address"] = address
    

290 nan


C:\Users\user\AppData\Local\Temp/ipykernel_15772/596618866.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  qry = browser.find_element_by_id("searchboxinput")
C:\Users\user\AppData\Local\Temp/ipykernel_15772/596618866.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = browser.find_element_by_id("searchbox-searchbutton")


291 nan
292 nan
293 nan
294 nan
295 nan
296 nan
297 nan
298 nan
299 nan
300 nan
301 nan
302 nan
303 nan
304 nan
305 nan
306 nan
307 nan
308 nan
309 nan
310 nan
311 nan
312 nan
313 nan
314 nan
315 nan
316 nan
317 nan
318 nan
319 nan
320 nan
321 nan
322 nan
323 nan
324 nan
325 nan
326 nan
327 nan
328 nan
329 nan
330 nan
331 nan
332 nan
333 nan
334 nan
335 nan
336 nan
337 nan
338 nan
339 nan
340 nan
341 nan
342 nan
343 nan
344 nan
345 nan
346 nan
347 nan
348 nan
349 nan
350 nan
351 nan
352 nan
353 nan
354 nan
355 nan
356 nan
357 nan
358 nan
359 nan
360 nan
361 nan
362 nan
363 nan
364 nan
365 nan
366 nan
367 nan
368 nan
369 nan
370 nan
371 nan
372 nan
373 nan
374 nan
375 nan
376 nan
377 nan
378 nan
379 nan
380 nan
381 nan
382 nan
383 nan
384 nan
385 nan
386 nan
387 nan
388 nan
389 nan
390 nan
391 nan
392 nan
393 nan
394 nan
395 nan
396 nan
397 nan
398 nan
399 nan
400 nan
401 nan
402 nan
403 nan
404 nan
405 nan
406 nan
407 nan
408 nan
409 nan
410 nan
411 nan
412 nan
413 nan
414 nan
415 nan


In [53]:
#export the file, but still have to manually adjust it.
df1.to_csv("gmap_missing.csv")